# Todo
- [x] take care of loading utils in right fodler (chatgpt)
- [ ] check for better way to only take cortex from cifti file
- [ ] write in readme of RetinoMaps how you generate the files of mask and put the code in github
- [x] get code for loading 91k
- [x] load single subject 91k resample GLM inter-task output
- [x] load single concatenated 91k resting state (xcpd output)
- [x] load 91k atlas to mask concatenated 91k resting state per macro-region seed (use leaveout -to avoid seed autocorrelation)
- [x] target: mask resting state with leaveout macro-region seed
- [x] seed: create 12 seed regions masked resting state timeseries
- [ ] correlate and get slope/r/stder
- [ ] correct for multiple test for significance
- [ ] median avg significant seed/target vertices to end with 12 maps for 12 seeds of only significant correlations
- [ ] plot 12 flatmaps for 12 seeds
- [ ] make winner take all plot

# Reference code
- load surface files utils
  - https://github.com/mszinte/pRF_analysis/blob/main/analysis_code/utils/surface_utils.py
    - loading: `load_surface`
    - saving: `make_surface_image`
- stats line codes :
  - usage: https://github.com/mszinte/pRF_analysis/blob/main/analysis_code/postproc/prf/postfit/compute_css_stats.py
  - utils: https://github.com/mszinte/pRF_analysis/blob/main/analysis_code/utils/maths_utils.py
  - correlation: `linear_regression_surf`
- drawing codes|
  - move back to 170k
  - or create 91k pycortex subject and utils
  - or use workbench

In [4]:
# Load individual subject 91k resample GLM inter-task output
import json
import numpy as np
import os, sys

sys.path.append(os.path.abspath('/home/mszinte/disks/meso_H/projects/pRF_analysis/analysis_code/utils/'))
from surface_utils import load_surface, make_surface_image
from maths_utils import linear_regression_surf

In [10]:
# Define settings
settings_filepath = '/home/mszinte/disks/meso_H/projects/pRF_analysis/RetinoMaps/settings.json'
with open(settings_filepath, 'r') as file:
    settings = json.load(file)

subjects = settings['subjects']
subject = subjects[0]
task_names = settings['task_names']
task_name = task_names[0]
rois = settings['rois']
fdr_alpha = settings['fdr_alpha']

# Define finename
main_data = '/home/mszinte/disks/meso_S/data'
project = settings['project_name']
seed_folder = 'derivatives/pp_data'
atlas_folder = '/home/mszinte/disks/meso_H/projects/pRF_analysis/analysis_code/atlas/macro_regions'

In [7]:
# Load resting-state dense timeseries
timeseries_fn = f'{main_data}/{project}/{seed_folder}/{subject}/91k/{task_name}/timeseries/{subject}_ses-01_task-{task_name}_space-fsLR_den-91k_desc-denoised_bold.dtseries.nii'
timeseries_img, timeseries_data = load_surface(timeseries_fn)
timeseries_data = timeseries_data[:, 0:64984]
print(f'Timeseries data shape: {timeseries_data.shape}')

resting-state data shape: (996, 64984)


In [8]:
# Load seed mask file as array (12,64984) from V1 to mPCS
for roi_num, roi in enumerate(rois):
    seed_mask_lh_fn = f'{main_data}/{project}/{seed_folder}/{subject}/91k/{task_name}/seed/{subject}_91k_intertask_Sac_Pur_vision-pursuit-saccade_lh_{roi}.shape.gii'
    seed_mask_lh_img, seed_mask_lh_data = load_surface(seed_mask_lh_fn)
    seed_mask_rh_fn = f'{main_data}/{project}/{seed_folder}/{subject}/91k/{task_name}/seed/{subject}_91k_intertask_Sac_Pur_vision-pursuit-saccade_rh_{roi}.shape.gii'
    seed_mask_rh_img, seed_mask_rh_data = load_surface(seed_mask_rh_fn)

    roi_seed_mask_data = np.hstack((seed_mask_lh_data, seed_mask_rh_data))

    if roi_num == 0:
        seed_mask_data = roi_seed_mask_data
    else:
        seed_mask_data = np.vstack((seed_mask_data, roi_seed_mask_data))

print(f'Seed mask data shape: {seed_mask_data.shape}')

Seed mask data shape: (12, 64984)


In [11]:
# Load leaveout macro-region target masks as array (12,64984) from V1 to mPCS
for roi_num, roi in enumerate(rois):
    leaveout_target_mask_lh_fn = f'{atlas_folder}/atlas-Glasser_space-fsLR_den-32k_filtered_ROIs_discarded_lh_{roi}.shape.gii'
    leaveout_target_mask_lh_img, leaveout_target_mask_lh_data = load_surface(leaveout_target_mask_lh_fn)
    leaveout_target_mask_rh_fn = f'{atlas_folder}/atlas-Glasser_space-fsLR_den-32k_filtered_ROIs_discarded_rh_{roi}.shape.gii'
    leaveout_target_mask_rh_img, leaveout_target_mask_rh_data = load_surface(leaveout_target_mask_rh_fn)

    roi_leaveout_target_mask_data = np.hstack((leaveout_target_mask_lh_data, leaveout_target_mask_rh_data))

    if roi_num == 0:
        leaveout_target_mask_data = roi_leaveout_target_mask_data
    else:
        leaveout_target_mask_data = np.vstack((leaveout_target_mask_data, roi_leaveout_target_mask_data))

print(f'Leave-out target mask data shape: {leaveout_target_mask_data.shape}')

Leave-out target mask data shape: (12, 64984)


In [7]:
# Correlation
for roi_num, roi in enumerate(rois):

    # solve problem with target data being too few vertices for some reason
    seed_data = timeseries_data[:, seed_mask_data[roi_num,:].astype(bool)]
    target_data = timeseries_data[:, leaveout_target_mask_data[roi_num,:].astype(bool)]

    
    results = linear_regression_surf(bold_signal=target_data[:,0], 
                                     model_prediction=seed_data[:,0], 
                                     correction='fdr_tsbh', 
                                     alpha=fdr_alpha)
    fff
    # Save results
    # prf_deriv_dir = "{}/{}/{}/prf/prf_derivatives".format(
    #     pp_dir, subject, format_)
    # stat_prf_loo_fn = prf_pred_loo_fn.split('/')[-1].replace('pred_css', 'stats')
    # stat_prf_loo_img = make_surface_image(data=results, 
    #                                       source_img=bold_img, 
    #                                       maps_names=maps_names)
    # print('Saving: {}/{}'.format(prf_deriv_dir, stat_prf_loo_fn))


    

TypeError: list indices must be integers or slices, not tuple